## Feature Selection

In [135]:
import os
import re
import nltk
import numpy as np
import pandas as pd
from sklearn import feature_extraction
from tqdm import tqdm

In [78]:
_wnl = nltk.WordNetLemmatizer()


def normalize_word(w):
    return _wnl.lemmatize(w).lower()


def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]


def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric

    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()


def remove_stopwords(l):
    # Removes stopwords from a list of tokens
    return [w for w in l if w not in feature_extraction.text.ENGLISH_STOP_WORDS]


def gen_or_load_feats(feat_fn, headlines, bodies, feature_file):
    if not os.path.isfile(feature_file):
        feats = feat_fn(headlines, bodies)
        np.save(feature_file, feats)

    return np.load(feature_file)

In [79]:
def word_overlap_features(headlines, bodies):
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        clean_headline = get_tokenized_lemmas(clean_headline)
        clean_body = get_tokenized_lemmas(clean_body)
        features = [
            len(set(clean_headline).intersection(clean_body)) / float(len(set(clean_headline).union(clean_body)))]
        X.append(features)
    return X

In [80]:

def refuting_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        # 'refute',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_headline = get_tokenized_lemmas(clean_headline)
        features = [1 if word in clean_headline else 0 for word in _refuting_words]
        X.append(features)
    return X

In [81]:
def polarity_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]

    def calculate_polarity(text):
        tokens = get_tokenized_lemmas(text)
        return sum([t in _refuting_words for t in tokens]) % 2
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        features = []
        features.append(calculate_polarity(clean_headline))
        features.append(calculate_polarity(clean_body))
        X.append(features)
    return np.array(X)

In [82]:

def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def chargrams(input, n):
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output

In [83]:

def append_chargrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in chargrams(" ".join(remove_stopwords(text_headline.split())), size)]
    grams_hits = 0
    grams_early_hits = 0
    grams_first_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
        if gram in text_body[:100]:
            grams_first_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    features.append(grams_first_hits)
    return features


def append_ngrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in ngrams(text_headline, size)]
    grams_hits = 0
    grams_early_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    return features

In [84]:

def hand_features(headlines, bodies):

    def binary_co_occurence(headline, body):
        # Count how many times a token in the title
        # appears in the body text.
        bin_count = 0
        bin_count_early = 0
        for headline_token in clean(headline).split(" "):
            if headline_token in clean(body):
                bin_count += 1
            if headline_token in clean(body)[:255]:
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def binary_co_occurence_stops(headline, body):
        # Count how many times a token in the title
        # appears in the body text. Stopwords in the title
        # are ignored.
        bin_count = 0
        bin_count_early = 0
        for headline_token in remove_stopwords(clean(headline).split(" ")):
            if headline_token in clean(body):
                bin_count += 1
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def count_grams(headline, body):
        # Count how many times an n-gram of the title
        # appears in the entire body, and intro paragraph

        clean_body = clean(body)
        clean_headline = clean(headline)
        features = []
        features = append_chargrams(features, clean_headline, clean_body, 2)
        features = append_chargrams(features, clean_headline, clean_body, 8)
        features = append_chargrams(features, clean_headline, clean_body, 4)
        features = append_chargrams(features, clean_headline, clean_body, 16)
        features = append_ngrams(features, clean_headline, clean_body, 2)
        features = append_ngrams(features, clean_headline, clean_body, 3)
        features = append_ngrams(features, clean_headline, clean_body, 4)
        features = append_ngrams(features, clean_headline, clean_body, 5)
        features = append_ngrams(features, clean_headline, clean_body, 6)
        return features

    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        X.append(binary_co_occurence(headline, body)
                 + binary_co_occurence_stops(headline, body)
                 + count_grams(headline, body))


    return X

## Fake News Challenge

In [85]:
import sys
import numpy as np
import random
import re
import argparse

from collections import defaultdict
from sklearn.ensemble import GradientBoostingClassifier
from csv import DictReader

#Import from other files
from utils.dataset import DataSet
from utils.generate_test_splits import kfold_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission
from utils.system import parse_params, check_version

### Reading Dataset

In [86]:
class DataSet():
    def __init__(self, name="train", path="fnc-1"):
        self.path = path

        print("Reading dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"

        self.stances = self.read(stances)
        
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])
        
        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))



    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "rt", encoding='utf-8-sig') as table:
            r = DictReader(table)
            for line in r:
                rows.append(line)
        return rows

### Generate test splits 

In [87]:
def generate_hold_out_split (dataset, training = 0.8, base_dir="splits"):
    r = random.Random()
    r.seed(1489215)

    article_ids = list(dataset.articles.keys())  # get a list of article ids
    r.shuffle(article_ids)  # and shuffle that list


    training_ids = article_ids[:int(training * len(article_ids))]
    hold_out_ids = article_ids[int(training * len(article_ids)):]

    # write the split body ids out to files for future use
    with open(base_dir+ "/"+ "training_ids.txt", "w+") as f:
        f.write("\n".join([str(id) for id in training_ids]))

    with open(base_dir+ "/"+ "hold_out_ids.txt", "w+") as f:
        f.write("\n".join([str(id) for id in hold_out_ids]))



def read_ids(file,base):
    ids = []
    with open(base+"/"+file,"r") as f:
        for line in f:
           ids.append(int(line))
        return ids


def kfold_split(dataset, training = 0.8, n_folds = 10, base_dir="splits"):
    if not (os.path.exists(base_dir+ "/"+ "training_ids.txt")
            and os.path.exists(base_dir+ "/"+ "hold_out_ids.txt")):
        generate_hold_out_split(dataset,training,base_dir)

    training_ids = read_ids("training_ids.txt", base_dir)
    hold_out_ids = read_ids("hold_out_ids.txt", base_dir)

    folds = []
    for k in range(n_folds):
        folds.append(training_ids[int(k*len(training_ids)/n_folds):int((k+1)*len(training_ids)/n_folds)])

    return folds,hold_out_ids


def get_stances_for_folds(dataset,folds,hold_out):
    stances_folds = defaultdict(list)
    stances_hold_out = []
    for stance in dataset.stances:
        if stance['Body ID'] in hold_out:
            stances_hold_out.append(stance)
        else:
            fold_id = 0
            for fold in folds:
                if stance['Body ID'] in fold:
                    stances_folds[fold_id].append(stance)
                fold_id += 1

    return stances_folds,stances_hold_out

### Score

In [88]:
#Adapted from https://github.com/FakeNewsChallenge/fnc-1/blob/master/scorer.py
#Original credit - @bgalbraith

LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


def report_score(actual,predicted):
    score,cm = score_submission(actual,predicted)
    best_score, _ = score_submission(actual,actual)

    print_confusion_matrix(cm)
    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score


if __name__ == "__main__":
    actual = [0,0,0,0,1,1,0,3,3]
    predicted = [0,0,0,0,1,1,2,3,3]

    report_score([LABELS[e] for e in actual],[LABELS[e] for e in predicted])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     4     |     0     |     1     |     0     |
-------------------------------------------------------------
| disagree  |     0     |     2     |     0     |     0     |
-------------------------------------------------------------
|  discuss  |     0     |     0     |     0     |     0     |
-------------------------------------------------------------
| unrelated |     0     |     0     |     0     |     2     |
-------------------------------------------------------------
Score: 6.75 out of 7.5	(90.0%)


### Systems

In [89]:
def parse_params():
    parser = argparse.ArgumentParser(description='FakeNewsChallenge fnc-1-baseline')
    parser.add_argument('-c', '--clean-cache', action='store_true', default=False, help="clean cache files")
#    params = parser.parse_args()
#    if not params.clean_cache:
#        return

    dr = "features"
    for f in os.listdir(dr):
        if re.search('\.npy$', f):
            fname = os.path.join(dr, f)
            os.remove(fname)
    for f in ['hold_out_ids.txt', 'training_ids.txt']:
        fname = os.path.join('splits', f)
        if os.path.isfile(fname):
            os.remove(fname)
    print("All clear")

def check_version():
    if sys.version_info.major < 3:
        sys.stderr.write('Please use Python version 3 and above\n')
        sys.exit(1)

#### Generate Features

In [90]:
def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "features/overlap."+name+".npy")
    X_refuting = gen_or_load_feats(refuting_features, h, b, "features/refuting."+name+".npy")
    X_polarity = gen_or_load_feats(polarity_features, h, b, "features/polarity."+name+".npy")
    X_hand = gen_or_load_feats(hand_features, h, b, "features/hand."+name+".npy")

    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    return X,y



In [101]:
if __name__ == "__main__":
    check_version()
    parse_params()


    #Load the training dataset and generate folds
    d = DataSet()
    folds,hold_out = kfold_split(d,n_folds=10)
    fold_stances, hold_out_stances = get_stances_for_folds(d,folds,hold_out)
    
        # Load the competition dataset
    competition_dataset = DataSet("competition_test")
    X_competition, y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")

    Xs = dict()
    ys = dict()

    # Load/Precompute all features now
    X_holdout,y_holdout = generate_features(hold_out_stances,d,"holdout")
    for fold in fold_stances:
        Xs[fold],ys[fold] = generate_features(fold_stances[fold],d,str(fold))


    best_score = 0
    best_fold = None
    
# # Classifier for each fold
# ## Uncomment this for loop code when you run this notebook for the first time. Takes over 30 mins to run.
#     for fold in fold_stances:
#         ids = list(range(len(folds)))
#         del ids[fold]

#         X_train = np.vstack(tuple([Xs[i] for i in ids]))
#         y_train = np.hstack(tuple([ys[i] for i in ids]))

#         X_test = Xs[fold]
#         y_test = ys[fold]

#         clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=False)
#         clf.fit(X_train, y_train)

#         predicted = [LABELS[int(a)] for a in clf.predict(X_test)]
#         actual = [LABELS[int(a)] for a in y_test]

#         fold_score, _ = score_submission(actual, predicted)
#         max_fold_score, _ = score_submission(actual, actual)

#         score = fold_score/max_fold_score

#         print("Score for fold "+ str(fold) + " was - " + str(score))
#         if score > best_score:
#             best_score = score
#             best_fold = clf
#-----------------



    #Run on Holdout set and report the final score on the holdout set
    predicted = [LABELS[int(a)] for a in best_fold.predict(X_holdout)]
    actual = [LABELS[int(a)] for a in y_holdout]

    print("Scores on the dev set")
    report_score(actual,predicted)
    print("")
    print("")

    #Run on competition dataset
    predicted = [LABELS[int(a)] for a in best_fold.predict(X_competition)]
    actual = [LABELS[int(a)] for a in y_competition]

    print("Scores on the test set")
    report_score(actual,predicted)
            

All clear
Reading dataset
Total stances: 49972
Total bodies: 1683


Reading dataset
Total stances: 25413
Total bodies: 904


0it [00:00, ?it/s]



35it [00:00, 338.25it/s]



66it [00:00, 328.56it/s]



104it [00:00, 342.11it/s]



143it [00:00, 351.79it/s]



178it [00:00, 349.37it/s]



214it [00:00, 351.37it/s]



246it [00:00, 323.48it/s]



277it [00:00, 301.34it/s]



306it [00:00, 267.75it/s]



333it [00:01, 268.03it/s]



370it [00:01, 289.79it/s]



400it [00:01, 287.96it/s]



441it [00:01, 315.96it/s]



476it [00:01, 324.74it/s]



510it [00:01, 322.81it/s]



550it [00:01, 341.55it/s]



585it [00:01, 334.72it/s]



620it [00:01, 337.61it/s]



656it [00:02, 338.21it/s]



691it [00:02, 324.51it/s]



730it [00:02, 341.60it/s]



765it [00:02, 320.11it/s]



800it [00:02, 326.68it/s]



834it [00:02, 318.78it/s]



867it [00:02, 309.81it/s]



902it [00:02, 320.31it/s]



935it [00:02, 305.62it/s]



973it [00:03, 322.47it/s]



1009it [00:03, 330.59it/s]



1046it [00:03, 340.61it/s]



1081it [00:03, 286.64it/s]



1112it [00:03, 239.42it/s]



1139it [00:03, 159.35it/s]



1161it [00:04, 131

13272it [00:43, 382.40it/s]



13311it [00:43, 366.97it/s]



13349it [00:43, 361.39it/s]



13386it [00:43, 349.45it/s]



13422it [00:43, 343.02it/s]



13457it [00:43, 343.10it/s]



13497it [00:44, 354.98it/s]



13537it [00:44, 366.26it/s]



13574it [00:44, 361.84it/s]



13611it [00:44, 348.35it/s]



13647it [00:44, 338.12it/s]



13682it [00:44, 317.73it/s]



13715it [00:44, 316.11it/s]



13749it [00:44, 322.02it/s]



13782it [00:44, 308.72it/s]



13818it [00:45, 321.71it/s]



13851it [00:45, 310.22it/s]



13883it [00:45, 304.25it/s]



13918it [00:45, 315.98it/s]



13950it [00:45, 307.40it/s]



13988it [00:45, 325.35it/s]



14021it [00:45, 318.32it/s]



14054it [00:45, 293.32it/s]



14089it [00:45, 306.68it/s]



14125it [00:46, 318.46it/s]



14158it [00:46, 318.50it/s]



14192it [00:46, 322.66it/s]



14233it [00:46, 342.19it/s]



14268it [00:46, 338.68it/s]



14307it [00:46, 352.34it/s]



14346it [00:46, 362.77it/s]



14383it [00:46, 354.82it/s]



14419it 

17038it [00:02, 6029.78it/s]



17642it [00:02, 5979.68it/s]



18242it [00:03, 5985.08it/s]



18873it [00:03, 6078.87it/s]



19482it [00:03, 6011.72it/s]



20084it [00:03, 5926.99it/s]



20678it [00:03, 5764.59it/s]



21313it [00:03, 5926.96it/s]



21908it [00:03, 5930.02it/s]



22524it [00:03, 5996.00it/s]



23125it [00:03, 5972.38it/s]



23748it [00:03, 6047.35it/s]



24354it [00:04, 6022.45it/s]



24963it [00:04, 6041.75it/s]



25413it [00:04, 6019.15it/s]



0it [00:00, ?it/s]



40it [00:00, 376.93it/s]



76it [00:00, 369.69it/s]



113it [00:00, 369.57it/s]



147it [00:00, 358.52it/s]



183it [00:00, 357.91it/s]



225it [00:00, 372.04it/s]



258it [00:00, 355.44it/s]



291it [00:00, 332.59it/s]



323it [00:00, 320.22it/s]



361it [00:01, 335.82it/s]



397it [00:01, 338.11it/s]



435it [00:01, 347.07it/s]



470it [00:01, 347.40it/s]



505it [00:01, 324.08it/s]



541it [00:01, 334.03it/s]



575it [00:01, 308.01it/s]



613it [00:01, 325.69it/s]



654it [

...

469it [00:01, 331.33it/s]



503it [00:01, 317.10it/s]



535it [00:01, 314.43it/s]



569it [00:01, 320.10it/s]



602it [00:01, 314.66it/s]



634it [00:01, 302.37it/s]



665it [00:02, 291.93it/s]



696it [00:02, 296.98it/s]



733it [00:02, 315.05it/s]



767it [00:02, 320.27it/s]



803it [00:02, 330.67it/s]



837it [00:02, 321.69it/s]



870it [00:02, 308.95it/s]



902it [00:02, 289.84it/s]



943it [00:02, 317.56it/s]



985it [00:03, 340.53it/s]



1022it [00:03, 348.58it/s]



1058it [00:03, 344.63it/s]



1094it [00:03, 334.14it/s]



1128it [00:03, 322.54it/s]



1161it [00:03, 311.73it/s]



1199it [00:03, 328.30it/s]



1233it [00:03, 324.72it/s]



1266it [00:03, 313.12it/s]



1305it [00:04, 329.42it/s]



1340it [00:04, 332.66it/s]



1374it [00:04, 313.70it/s]



1406it [00:04, 301.45it/s]



1437it [00:04, 294.87it/s]



1467it [00:04, 286.22it/s]



1496it [00:04, 279.31it/s]



1533it [00:04, 299.56it/s]



1566it [00:04, 308.00it/s]



1599it [00:05, 310.09it/s]

4099it [00:16, 235.24it/s]



4123it [00:16, 209.51it/s]



4148it [00:16, 216.82it/s]



4174it [00:16, 226.31it/s]



4198it [00:16, 230.01it/s]



4224it [00:16, 237.83it/s]



4249it [00:16, 241.20it/s]



4273it [00:16, 253.75it/s]



0it [00:00, ?it/s]



40it [00:00, 398.68it/s]



70it [00:00, 362.40it/s]



102it [00:00, 346.27it/s]



134it [00:00, 336.77it/s]



163it [00:00, 320.33it/s]



196it [00:00, 320.24it/s]



234it [00:00, 336.09it/s]



266it [00:00, 329.42it/s]



298it [00:00, 326.30it/s]



330it [00:01, 302.81it/s]



360it [00:01, 291.37it/s]



393it [00:01, 301.95it/s]



424it [00:01, 286.46it/s]



453it [00:01, 280.89it/s]



490it [00:01, 301.89it/s]



521it [00:01, 301.02it/s]



557it [00:01, 315.32it/s]



589it [00:01, 278.87it/s]



618it [00:02, 260.94it/s]



646it [00:02, 257.66it/s]



673it [00:02, 254.53it/s]



704it [00:02, 268.65it/s]



739it [00:02, 287.73it/s]



774it [00:02, 302.59it/s]



806it [00:02, 272.35it/s]



835it [00:02, 2

3338it [00:13, 236.54it/s]



3368it [00:13, 249.67it/s]



3394it [00:14, 237.82it/s]



3421it [00:14, 243.52it/s]



3450it [00:14, 255.52it/s]



3476it [00:14, 238.77it/s]



3501it [00:14, 233.77it/s]



3525it [00:14, 227.30it/s]



3552it [00:14, 238.49it/s]



3577it [00:14, 232.47it/s]



3602it [00:14, 236.61it/s]



3626it [00:15, 233.72it/s]



3650it [00:15, 233.39it/s]



3674it [00:15, 224.07it/s]



3697it [00:15, 206.78it/s]



3722it [00:15, 216.97it/s]



3745it [00:15, 208.74it/s]



3767it [00:15, 205.48it/s]



3789it [00:15, 207.33it/s]



3819it [00:15, 224.43it/s]



3849it [00:16, 241.74it/s]



3874it [00:16, 240.08it/s]



3901it [00:16, 246.35it/s]



3927it [00:16, 249.13it/s]



3953it [00:16, 241.40it/s]



3978it [00:16, 237.08it/s]



4006it [00:16, 245.27it/s]



4031it [00:16, 232.39it/s]



4039it [00:16, 240.06it/s]



0it [00:00, ?it/s]



26it [00:00, 255.12it/s]



55it [00:00, 262.84it/s]



90it [00:00, 282.80it/s]



122it [00:00, 291.66it/s

1973it [00:05, 317.97it/s]



2007it [00:05, 323.11it/s]



2042it [00:06, 326.24it/s]



2075it [00:06, 312.65it/s]



2107it [00:06, 304.94it/s]



2141it [00:06, 307.53it/s]



2174it [00:06, 313.82it/s]



2206it [00:06, 314.44it/s]



2238it [00:06, 312.05it/s]



2275it [00:06, 327.12it/s]



2314it [00:06, 342.97it/s]



2349it [00:07, 328.87it/s]



2383it [00:07, 314.62it/s]



2421it [00:07, 330.75it/s]



2457it [00:07, 337.42it/s]



2492it [00:07, 303.48it/s]



2532it [00:07, 326.65it/s]



2566it [00:07, 319.36it/s]



2601it [00:07, 326.30it/s]



2636it [00:07, 323.86it/s]



2669it [00:08, 311.23it/s]



2710it [00:08, 326.76it/s]



2748it [00:08, 340.85it/s]



2786it [00:08, 350.68it/s]



2822it [00:08, 336.54it/s]



2857it [00:08, 334.99it/s]



2891it [00:08, 313.43it/s]



2923it [00:08, 293.00it/s]



2953it [00:08, 286.20it/s]



2989it [00:09, 304.41it/s]



3029it [00:09, 327.17it/s]



3063it [00:09, 324.44it/s]



3097it [00:09, 324.62it/s]



3130it [00

Score for fold 6 was - 0.7740591783970123
Score for fold 0 was - 0.790634959548909
Score for fold 7 was - 0.8065337293169283
Score for fold 5 was - 0.7642120765832106
Score for fold 2 was - 0.8175341669089852
Score for fold 8 was - 0.820952380952381
Score for fold 9 was - 0.7873290538654758
Score for fold 3 was - 0.8108217514505465
Score for fold 1 was - 0.7939656376588909
Score for fold 4 was - 0.7953927600515095
Scores on the dev set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    118    |     3     |    556    |    85     |
-------------------------------------------------------------
| disagree  |    14     |     3     |    130    |    15     |
-------------------------------------------------------------
|  discuss  |    58     |     5     |   1527    |    210    |
-------------------------------------------------------------
| 

In [268]:
def generate_fn_features(dataset,name):
    h, b = [],[]

    for d in dataset:
        h.append(d[0]) #title
        b.append(d[1]) #text

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "features/overlap."+name+".npy")
    X_refuting = gen_or_load_feats(refuting_features, h, b, "features/refuting."+name+".npy")
    X_polarity = gen_or_load_feats(polarity_features, h, b, "features/polarity."+name+".npy")
    X_hand = gen_or_load_feats(hand_features, h, b, "features/hand."+name+".npy")

    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    return X

In [287]:
df_Orig = pd.read_csv('fake_real_dataset.csv')
df = pd.read_csv('fake_real_dataset.csv')

In [288]:
def text_column(tuple1):
    if(pd.notna(tuple1[1])):
        if(tuple1[1].strip(' \t\n\r') == ''):
            return tuple1[0]
        else:
            return tuple1[1] 
    else:
            return tuple1[1]

In [289]:
df['text'] = df[['title', 'text']].apply(text_column, axis=1)

In [306]:
def title_column(tuple1):
    if(pd.notna(tuple1[0])):
        if(tuple1[0].strip(' \t\n\r') == ''):
            return re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", tuple1[1])
        else:
            return re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", tuple1[0])
    else:
            return re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", tuple1[0])

In [300]:
fn_dataset = df[['title', 'text']]
fn_dataset.shape

(100, 2)

In [301]:
fn_dataset = fn_dataset.values
len(fn_dataset)

100

In [302]:
X_fn = generate_fn_features(fn_dataset, "FakeNewsStance")

In [303]:
X_fn.shape

(10, 44)

In [276]:
   fn_predicted = [LABELS[int(a)] for a in best_fold.predict(X_fn)]

In [277]:
fn_predicted

['discuss',
 'discuss',
 'discuss',
 'discuss',
 'unrelated',
 'discuss',
 'agree',
 'discuss',
 'discuss',
 'discuss']

In [265]:
#     # Load the Fake News Original dataset
# #    competition_dataset = DataSet("competition_test")
# #    X_competition, y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")
    
# #    dataset = ["title","text"]
    
#     fn_dataset = df[['title', 'text']]
    
#     fn_dataset = fn_dataset.values
    
#     X_fn = generate_fn_features(fn_dataset, "fakenews")
    
#     # Predict on Fake News Original dataset
#     fn_predicted = [LABELS[int(a)] for a in best_fold.predict(X_fn)]